In [1]:
import gen3schemadev as g

In [2]:
lipid_schema = g.utils.load_yaml("../output/lipidomics_file.yaml")
rule_val = g.validators.rule_validator.RuleValidator(lipid_schema)
rule_val.data_file_props_need_data_props()

ValueError: Schema 'lipidomics_file' with category 'data_file' must include properties 'data_type', 'data_format', and 'data_category'. Please add these properties to the 'properties' section.

In [ ]:
sdfsa

In [ ]:
import gen3schemadev as g
# rs = g.utils.resolve_schema(schema_path="../tests/gen3_schema/testing/schema_dev_pass.json")
rs = g.utils.resolve_schema(schema_path="../tests/gen3_schema/examples/json/schema_dev.json")

In [ ]:
rs

In [ ]:
g.utils.write_json(rs, "../tests/gen3_schema/examples/json/schema_dev_resolved.json")

In [ ]:
type(rs)

In [ ]:
g.utils.bundled_schema_to_list_dict(rs)